In [1]:
!pip install evaluate
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 5.2 MB/s eta 0:00:00


# Data Load

In [2]:
from datasets import load_dataset

# text_1 -> ind, text_2 -> mad
mdr = load_dataset("indonlp/NusaX-MT",name="ind-mad") 
mdr

README.md:   0%|          | 0.00/5.64k [00:00<?, ?B/s]

NusaX-MT.py:   0%|          | 0.00/5.69k [00:00<?, ?B/s]

The repository for indonlp/NusaX-MT contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/indonlp/NusaX-MT.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text_1', 'text_2', 'text_1_lang', 'text_2_lang'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['id', 'text_1', 'text_2', 'text_1_lang', 'text_2_lang'],
        num_rows: 100
    })
    test: Dataset({
        features: ['id', 'text_1', 'text_2', 'text_1_lang', 'text_2_lang'],
        num_rows: 400
    })
})

In [3]:
train = mdr["train"]
val = mdr["validation"]
test = mdr["test"]

In [4]:
from transformers.utils import move_cache
move_cache()


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [5]:
from transformers import AutoTokenizer

# https://huggingface.co/indonlp/cendol-mt5-small-inst
checkpoint = "LazarusNLP/IndoNanoT5-base" # GANTI MODELNYA
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [6]:
prefix = "terjemahkan dari Bahasa Madura ke Bahasa Indonesia: " # nanti pas inference, perlu di append prefix nya!

def preprocess_function(examples):
    # Menambahkan prefix dan melakukan lowercasing pada input
    inputs = [prefix + example.lower() for example in examples["text_2"]]  # optional: .lower() jika perlu
    targets = [example.lower() for example in examples["text_1"]]  # optional: .lower() jika perlu
    
    # Tokenisasi
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    
    return model_inputs

In [7]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_val = val.map(preprocess_function, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [9]:
import evaluate
import numpy as np

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Post-process texts
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Compute BLEU score
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    # Calculate average generation length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = round(np.mean(prediction_lens), 4)
    
    return result

In [10]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [11]:
training_args = Seq2SeqTrainingArguments(
    output_dir="mad-indo_IndoNano-T5_25",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=28,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

#8e273a83f932e54c56230b4e303a2f335701ad7b

/tmp/ipykernel_23/239706790.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,4.269657,1.296358,15.560000
2,No log,3.874518,2.267369,16.590000
3,No log,3.483699,3.307245,17.020000
4,No log,3.327253,4.697349,16.330000
5,No log,3.136981,5.712170,16.690000
6,No log,3.019821,6.664392,17.360000
7,No log,2.964751,7.939129,17.400000
8,No log,2.924191,7.819902,17.420000
9,No log,2.888624,8.751884,17.510000
10,No log,2.869298,9.514506,17.570000


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=896, training_loss=1.8986884866442, metrics={'train_runtime': 570.9449, 'train_samples_per_second': 24.521, 'train_steps_per_second': 1.569, 'total_flos': 2342212524048384.0, 'train_loss': 1.8986884866442, 'epoch': 28.0})

In [12]:
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [13]:
test_results = trainer.evaluate(tokenized_test)
print("Test BLEU score:", test_results["eval_bleu"])

Test BLEU score: 11.37020697130227


In [14]:
test_results

{'eval_loss': 2.789724111557007,
 'eval_bleu': 11.37020697130227,
 'eval_gen_len': 17.29,
 'eval_runtime': 17.0276,
 'eval_samples_per_second': 23.491,
 'eval_steps_per_second': 1.468,
 'epoch': 28.0}

## SAVE AND LOAD

In [ ]:
# BUAT NGESAVE KE LOCAL!!!
local_dir = "cendol-mt5-small-inst"

model.save_pretrained(local_dir)
tokenizer.save_pretrained(local_dir)

# NANTI CARA NGE LOAD NYA GINI!!
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# model = AutoModelForSeq2SeqLM.from_pretrained(local_dir)
# tokenizer = AutoTokenizer.from_pretrained(local_dir)

## Inference

In [ ]:
test

In [ ]:
import random
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained(local_dir)
tokenizer = AutoTokenizer.from_pretrained(local_dir)

def translate_and_compare_samples(test_data, num_samples=5):
    samples = random.sample(range(len(test_data['text_1'])), num_samples)
    
    for idx in samples:
        source_text = test_data['text_1'][idx]
        target_text = test_data['text_2'][idx]
        text = f"terjemahkan dari Bahasa Indonesia ke Bahasa Madura: {source_text}"
        inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True)
        outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
        translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"Sampel {idx + 1}")
        print("Source Text (Indonesia):", source_text)
        print("Translated Text (Madura):", translated_text)
        print("Target Text (Expected Madura):", target_text)
        print("=" * 50)

translate_and_compare_samples(test, num_samples=5)


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(local_dir)
tokenizer = AutoTokenizer.from_pretrained(local_dir)

text = "terjemahkan dari Bahasa Indonesia ke Bahasa Madura: gimana kabarmu?"

inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True)
outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)

translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Translated text:", translated_text)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Inisialisasi model dan tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(local_dir)
tokenizer = AutoTokenizer.from_pretrained(local_dir)

# Teks masukan dalam bahasa Madura
text = "terjemahkan dari Bahasa Madura ke Bahasa Indonesia: Semmak bik hotel engkok nginep, pera' ejeleni ajelen soko, ediye bennyak sarah pelean kakananna, kenengngan se leber, ben masenneng"

# Tokenisasi input
inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True)

# Generasi terjemahan
outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)

# Dekode hasil terjemahan
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Translated text:", translated_text)

In [ ]:
# import random
# import pandas as pd
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# import sacrebleu

# # Inisialisasi model dan tokenizer
# model = AutoModelForSeq2SeqLM.from_pretrained(local_dir)
# tokenizer = AutoTokenizer.from_pretrained(local_dir)

# # Fungsi untuk menerjemahkan dan membandingkan hasil dengan skor BLEU
# def translate_and_compare_samples(test_data, num_samples=100):
#     # Ambil beberapa sampel acak dari data test
#     samples = random.sample(range(len(test_data['text_1'])), num_samples)

#     # List untuk menyimpan hasil yang akan dikonversi menjadi tabel
#     results = []

#     for idx in samples:
#         source_text = test_data['text_1'][idx]
#         target_text = test_data['text_2'][idx]
#         text = f"terjemahkan dari Bahasa Indonesia ke Bahasa Madura: {source_text}"
        
#         # Tokenisasi dan terjemahan
#         inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True)
#         outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
        
#         # Dekode hasil terjemahan
#         translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
#         # Hitung skor BLEU
#         bleu = sacrebleu.corpus_bleu([translated_text], [[target_text]]).score
        
#         # Tambahkan hasil ke list
#         results.append({
#             "Source Text (Indonesia)": source_text,
#             "Translated Text (Madura)": translated_text,
#             "Target Text (Expected Madura)": target_text,
#             "BLEU Score": bleu
#         })

#     # Konversi hasil ke dalam DataFrame pandas
#     df_results = pd.DataFrame(results)
#     return df_results

# # Panggil fungsi dan tampilkan hasil dalam bentuk tabel
# df_results = translate_and_compare_samples(test, num_samples=100)
# df_results


In [ ]:
# df_results.to_csv("translation_results_full.csv", index=False)